In [1]:
import speech_recognition as sr

r = sr.Recognizer()

language_code = {
  "kannada": "kn-IN",
  "gujarati": "gu-IN",
  "marathi": "mr-IN",
  "bengali": "bn-IN",
  "hindi": "hi-IN",
  "english": "en-IN"  #  for Indian English
}

chosen_language = "hindi"  # Based on user selection in frontend

print(f"Say something in {chosen_language}:")

with sr.Microphone() as source:
    audio = r.listen(source)
    print("Done!")

text = r.recognize_google(audio, language=language_code[chosen_language])

print(f"You said: {text}")
print(f"In englis: {r.recognize_google(audio)}")


Say something in hindi:
Done!
You said: चोली के पीछे क्या है चोली के पीछे
In englis: Choli Ke Piche Kya Hai Choli Ke Piche


In [2]:
import googletrans

print(googletrans.LANGUAGES)


{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [3]:
from googletrans import Translator

translator = Translator()
translated = translator.translate(text)

print(translated.text)

What's behind the bodice? What's behind the bodice?


In [23]:
import tensorflow as tf
from tqdm.notebook import tqdm
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import numpy as np
from tensorflow.keras.utils import unpack_x_y_sample_weight

In [24]:
# Load pre-trained DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [25]:
dataset = [
    ("Can you turn on the lights of room 1?","Light on room1","Turning on the lights of room1"),
("I'd like to have the lights of room 1 turned on, can you do that?","Light on room1","Turning on the lights of room1"),
("Is there a way to activate the lights for room 1?","Light on room1","Turning on the lights of room1"),
("Can you switch on the lights of room 1, please?","Light on room1","Turning on the lights of room1"),
("I need some light of room 1, could you help?","Light on room1","Turning on the lights of room1"),
("Let's illuminate room 1, shall we?","Light on room1","Turning on the lights of room1"),
("How about turning on the lights for room 1?","Light on room1","Turning on the lights of room1"),
("Can you activate the lights of room 1, please?","Light on room1","Turning on the lights of room1"),
("It's quite dark of room 1, could you turn on the lights?","Light on room1","Turning on the lights of room1"),
("Would you mind brightening up room 1?","Light on room1","Turning on the lights of room1"),
("Can you turn on the fan of room 1?","Fan on of room1","Fan for room1 is on. Enjoy the cool air."),
("How about switching on the fan for room 1?","Fan on of room1","Fan for room1 is on. Enjoy the cool air."),
("Can you activate the fan of room 1, please?","Fan on of room1","Fan for room1 is on. Enjoy the cool air."),
("Could you turn off all the lights, please?","Turn off all lights","All lights and fans are off now."),
("Can you switch off all the lights, please?","Turn off all lights","All lights and fans are off now."),
("Please turn off all the lights.","Turn off all lights","All lights and fans are off now."),
("Can you extinguish all lights?","Turn off all lights","All lights and fans are off now."),
("Turn off all lights, please.","Turn off all lights","All lights and fans are off now."),
("Can you deactivate all lights?","Turn off all lights","All lights and fans are off now."),
("I want to turn off all the lights.","Turn off all lights","All lights and fans are off now."),
("Is it possible to turn on the lights of room 2?","Light on room2","Turning on the lights of room2"),
("Can you switch on the lights of room 2?","Light on room2","Turning on the lights of room2"),
("How about turning on the lights for room 2?","Light on room2","Turning on the lights of room2"),
("Can you activate the lights of room 2,please?","Light on room2","Turning on the lights of room2"),
("It's quite dark of room 2, can you turn on the lights?","Light on room2","Turning on the lights of room2"),
("I need some light of room 2, could you help me?","Light on room2","Turning on the lights of room2"),
("Let's illuminate room 2, shall we?","Light on room2","Turning on the lights of room2"),
("Would you mind brightening up room 2?","Light on room2","Turning on the lights of room2"),
("Can you turn on the fan of room 2?","Fan on of room2","Fan for room2 is on. Enjoy the cool air."),
("How about switching on the fan for room 2?","Fan on of room2","Fan for room2 is on. Enjoy the cool air."),
("Can you activate the fan of room 2, please?","Fan on of room2","Fan for room2 is on. Enjoy the cool air."),
("Could you turn off all the lights of the kitchen, please?","Turn off kitchen lights","All kitchen lights are off now."),
("Please switch off all lights of the kitchen.","Turn off kitchen lights","All kitchen lights are off now."),
("Can you deactivate the lights of the kitchen?","Turn off kitchen lights","All kitchen lights are off now."),
("I want to turn off all the lights of the kitchen.","Turn off kitchen lights","All kitchen lights are off now."),
("Turn off all lights of the kitchen, please.","Turn off kitchen lights","All kitchen lights are off now."),
("Is it possible to turn on the bedroom lights?","Bedroom light on","Bedroom lights are on now."),
("Can you switch on the lights of the bedroom?","Bedroom light on","Bedroom lights are on now."),
("How about turning on the lights for the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you activate the lights of the bedroom,please?","Bedroom light on","Bedroom lights are on now."),
("It's quite dark of the bedroom, can you turn on the lights?","Bedroom light on","Bedroom lights are on now."),
("I need some light of the bedroom, could you help me?","Bedroom light on","Bedroom lights are on now."),
("Let's illuminate the bedroom, shall we?","Bedroom light on","Bedroom lights are on now."),
("Would you mind brightening up the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you turn on the fan of the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("How about switching on the fan for the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you activate the fan of the bedroom, please?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you turn on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Is it possible to switch on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Can you activate the TV of the living room, please?","TV on of living room","TV of the living room is now on."),
("It's quite dark of room 3, can you turn on the lights?","Turn off lights of room3","Lights of room3 are turned off now."),
("I need some light of room 3, could you help me?","Turn off lights of room3","Lights of room3 are turned off now."),
("Let's illuminate room 3,shall we?","Turn off lights of room3","Lights of room3 are turned off now."),
("Would you mind brightening up room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you turn on the fan of room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("How about switching on the fan for room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you activate the fan of room 3, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Could you turn off all the lights of the kitchen, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Please switch off all lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you deactivate the lights of the kitchen?","Turn off lights of room3","Lights of room3 are turned off now."),
("I want to turn off all the lights of the kitchen.","Turn off kitchen lights","All kitchen lights are off now."),
("Turn off all lights of the kitchen, please.","Turn off kitchen lights","All kitchen lights are off now."),
("Is it possible to turn on the bedroom lights?","Bedroom light on","Bedroom lights are on now."),
("Can you switch on the lights of the bedroom?","Bedroom light on","Bedroom lights are on now."),
("How about turning on the lights for the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you activate the lights of the bedroom, please?","Bedroom light on","Bedroom lights are on now."),
("It's quite dark of the bedroom, can you turn on the lights?","Bedroom light on","Bedroom lights are on now."),
("I need some light of the bedroom,could you help me?","Bedroom light on","Bedroom lights are on now."),
("Let's illuminate the bedroom, shall we?","Bedroom light on","Bedroom lights are on now."),
("Would you mind brightening up the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you turn on the fan of the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("How about switching on the fan for the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you activate the fan of the bedroom,please?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you turn on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Is it possible to switch on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Can you activate the TV of the living room, please?","TV on of living room","TV of the living room is now on."),
("It's quite dark of room 3, can you turn on the lights?","Turn off lights of room3","Lights of room3 are turned off now."),
("I need some light of room 3, could you help me?","Turn off lights of room3","Lights of room3 are turned off now."),
("Let's illuminate room 3, shall we?","Turn off lights of room3","Lights of room3 are turned off now."),
("Would you mind brightening up room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you turn on the fan of room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("How about switching on the fan for room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you activate the fan of room 3, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Could you turn off all the lights of the kitchen, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Please switch off all lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you deactivate the lights of the kitchen?","Turn off lights of room3","Lights of room3 are turned off now."),
("I want to turn off all the lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Turn off all lights of the kitchen, please.","Turn off lights of room3","Lights of room3 are turned off now."),
("Is it possible to turn on the bedroom lights?","Bedroom light on","Bedroom lights are on now."),
("Can you switch on the lights of the bedroom?","Bedroom light on","Bedroom lights are on now."),
("How about turning on the lights for the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you activate the lights of the bedroom, please?","Bedroom light on","Bedroom lights are on now."),
("It's quite dark of the bedroom, can you turn on the lights?","Bedroom light on","Bedroom lights are on now."),
("I need some light of the bedroom, could you help me?","Bedroom light on","Bedroom lights are on now."),
("Let's illuminate the bedroom, shall we?","Bedroom light on","Bedroom lights are on now."),
("Would you mind brightening up the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you turn on the fan of the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("How about switching on the fan for the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you activate the fan of the bedroom, please?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you turn on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Is it possible to switch on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Can you activate the TV of the living room,please?","TV on of living room","TV of the living room is now on."),
("It's quite dark of room 3, can you turn on the lights?","Turn off lights of room3","Lights of room3 are turned off now."),
("I need some light of room 3, could you help me?","Turn off lights of room3","Lights of room3 are turned off now."),
("Let's illuminate room 3, shall we?","Turn off lights of room3","Lights of room3 are turned off now."),
("Would you mind brightening up room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you turn on the fan of room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("How about switching on the fan for room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you activate the fan of room 3, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Could you turn off all the lights of the kitchen, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Please switch off all lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you deactivate the lights of the kitchen?","Turn off lights of room3","Lights of room3 are turned off now."),
("I want to turn off all the lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Turn off all lights of the kitchen, please.","Turn off lights of room3","Lights of room3 are turned off now."),
("Is it possible to turn on the bedroom lights?","Bedroom light on","Bedroom lights are on now."),
("Can you switch on the lights of the bedroom?","Bedroom light on","Bedroom lights are on now."),
("How about turning on the lights for the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you activate the lights of the bedroom, please?","Bedroom light on","Bedroom lights are on now."),
("It's quite dark of the bedroom, can you turn on the lights?","Bedroom light on","Bedroom lights are on now."),
("I need some light of the bedroom, could you help me?","Bedroom light on","Bedroom lights are on now."),
("Let's illuminate the bedroom, shall we?","Bedroom light on","Bedroom lights are on now."),
("Would you mind brightening up the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you turn on the fan of the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("How about switching on the fan for the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you activate the fan of the bedroom, please?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you turn on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Is it possible to switch on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Can you activate the TV of the living room, please?","TV on of living room","TV of the living room is now on."),
("It's quite dark of room 3, can you turn on the lights?","Turn off lights of room3","Lights of room3 are turned off now."),
("I need some light of room 3, could you help me?","Turn off lights of room3","Lights of room3 are turned off now."),
("Let's illuminate room 3, shall we?","Turn off lights of room3","Lights of room3 are turned off now."),
("Would you mind brightening up room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you turn on the fan of room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("How about switching on the fan for room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you activate the fan of room 3, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Could you turn off all the lights of the kitchen, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Please switch off all lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you deactivate the lights of the kitchen?","Turn off lights of room3","Lights of room3 are turned off now."),
("I want to turn off all the lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Turn off all lights of the kitchen, please.","Turn off lights of room3","Lights of room3 are turned off now."),
("Is it possible to turn on the bedroom lights?","Bedroom light on","Bedroom lights are on now."),
("Can you switch on the lights of the bedroom?","Bedroom light on","Bedroom lights are on now."),
("How about turning on the lights for the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you activate the lights of the bedroom, please?","Bedroom light on","Bedroom lights are on now."),
("It's quite dark of the bedroom, can you turn on the lights?","Bedroom light on","Bedroom lights are on now."),
("I need some light of the bedroom, could you help me?","Bedroom light on","Bedroom lights are on now."),
("Let's illuminate the bedroom, shall we?","Bedroom light on","Bedroom lights are on now."),
("Would you mind brightening up the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you turn on the fan of the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("How about switching on the fan for the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you activate the fan of the bedroom, please?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you turn on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Is it possible to switch on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Can you activate the TV of the living room,please?","TV on of living room","TV of the living room is now on."),
("It's quite dark of room 3, can you turn on the lights?","Turn off lights of room3","Lights of room3 are turned off now."),
("I need some light of room 3,could you help me?","Turn off lights of room3","Lights of room3 are turned off now."),
("Let's illuminate room 3, shall we?","Turn off lights of room3","Lights of room3 are turned off now."),
("Would you mind brightening up room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you turn on the fan of room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("How about switching on the fan for room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you activate the fan of room 3, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Could you turn off all the lights of the kitchen, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Please switch off all lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you deactivate the lights of the kitchen?","Turn off lights of room3","Lights of room3 are turned off now."),
("I want to turn off all the lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Turn off all lights of the kitchen, please.","Turn off lights of room3","Lights of room3 are turned off now."),
("Is it possible to turn on the bedroom lights?","Bedroom light on","Bedroom lights are on now."),
("Can you switch on the lights of the bedroom?","Bedroom light on","Bedroom lights are on now."),
("How about turning on the lights for the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you activate the lights of the bedroom, please?","Bedroom light on","Bedroom lights are on now."),
("It's quite dark of the bedroom, can you turn on the lights?","Bedroom light on","Bedroom lights are on now."),
("I need some light of the bedroom, could you help me?","Bedroom light on","Bedroom lights are on now."),
("Let's illuminate the bedroom, shall we?","Bedroom light on","Bedroom lights are on now."),
("Would you mind brightening up the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you turn on the fan of the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("How about switching on the fan for the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you activate the fan of the bedroom, please?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you turn on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Is it possible to switch on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Can you activate the TV of the living room, please?","TV on of living room","TV of the living room is now on."),
("It's quite dark of room 3, can you turn on the lights?","Turn off lights of room3","Lights of room3 are turned off now."),
("I need some light of room 3, could you help me?","Turn off lights of room3","Lights of room3 are turned off now."),
("Let's illuminate room 3, shall we?","Turn off lights of room3","Lights of room3 are turned off now."),
("Would you mind brightening up room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you turn on the fan of room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("How about switching on the fan for room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you activate the fan of room 3, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Could you turn off all the lights of the kitchen, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Please switch off all lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you deactivate the lights of the kitchen?","Turn off lights of room3","Lights of room3 are turned off now."),
("I want to turn off all the lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Turn off all lights of the kitchen, please.","Turn off lights of room3","Lights of room3 are turned off now."),
("Is it possible to turn on the bedroom lights?","Bedroom light on","Bedroom lights are on now."),
("Can you switch on the lights of the bedroom?","Bedroom light on","Bedroom lights are on now."),
("How about turning on the lights for the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you activate the lights of the bedroom, please?","Bedroom light on","Bedroom lights are on now."),
("It's quite dark of the bedroom, can you turn on the lights?","Bedroom light on","Bedroom lights are on now."),
("I need some light of the bedroom, could you help me?","Bedroom light on","Bedroom lights are on now."),
("Let's illuminate the bedroom, shall we?","Bedroom light on","Bedroom lights are on now."),
("Would you mind brightening up the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you turn on the fan of the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("How about switching on the fan for the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you activate the fan of the bedroom, please?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you turn on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Is it possible to switch on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Can you activate the TV of the living room, please?","TV on of living room","TV of the living room is now on."),
("It's quite dark of room 3, can you turn on the lights?","Turn off lights of room3","Lights of room3 are turned off now."),
("I need some light of room 3, could you help me?","Turn off lights of room3","Lights of room3 are turned off now."),
("Let's illuminate room 3, shall we?","Turn off lights of room3","Lights of room3 are turned off now."),
("Would you mind brightening up room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you turn on the fan of room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("How about switching on the fan for room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you activate the fan of room 3, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Could you turn off all the lights of the kitchen, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Please switch off all lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you deactivate the lights of the kitchen?","Turn off lights of room3","Lights of room3 are turned off now."),
("I want to turn off all the lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Turn off all lights of the kitchen, please.","Turn off lights of room3","Lights of room3 are turned off now."),
("Is it possible to turn on the bedroom lights?","Bedroom light on","Bedroom lights are on now."),
("Can you switch on the lights of the bedroom?","Bedroom light on","Bedroom lights are on now."),
("How about turning on the lights for the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you activate the lights of the bedroom, please?","Bedroom light on","Bedroom lights are on now."),
("It's quite dark of the bedroom, can you turn on the lights?","Bedroom light on","Bedroom lights are on now."),
("I need some light of the bedroom, could you help me?","Bedroom light on","Bedroom lights are on now."),
("Let's illuminate the bedroom, shall we?","Bedroom light on","Bedroom lights are on now."),
("Would you mind brightening up the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you turn on the fan of the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("How about switching on the fan for the bedroom?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you activate the fan of the bedroom, please?","Bedroom fan on","Bedroom fan is on now. Enjoy the comfort."),
("Can you turn on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Is it possible to switch on the TV of the living room?","TV on of living room","TV of the living room is now on."),
("Can you activate the TV of the living room, please?","TV on of living room","TV of the living room is now on."),
("It's quite dark of room 3 can you turn on the lights?","Turn off lights of room3","Lights of room3 are turned off now."),
("I need some light of room 3 could you help me?","Turn off lights of room3","Lights of room3 are turned off now."),
("Let's illuminate room 3, shall we?","Turn off lights of room3","Lights of room3 are turned off now."),
("Would you mind brightening up room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you turn on the fan of room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("How about switching on the fan for room 3?","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you activate the fan of room 3, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Could you turn off all the lights of the kitchen, please?","Turn off lights of room3","Lights of room3 are turned off now."),
("Please switch off all lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Can you deactivate the lights of the kitchen?","Turn off lights of room3","Lights of room3 are turned off now."),
("I want to turn off all the lights of the kitchen.","Turn off lights of room3","Lights of room3 are turned off now."),
("Turn off all lights of the kitchen, please.","Turn off lights of room3","Lights of room3 are turned off now."),
("Is it possible to turn on the bedroom lights?","Bedroom light on","Bedroom lights are on now."),
("Can you switch on the lights of the bedroom?","Bedroom light on","Bedroom lights are on now."),
("How about turning on the lights for the bedroom?","Bedroom light on","Bedroom lights are on now."),
("Can you activate the lights of the bedroom, please?","Bedroom light on","Bedroom lights are on now."),
("It's quite dark of the bedroom, can you turn on the lights?","Bedroom light on","Bedroom lights are on now."),
("I need some light of the bedroom, could you help me?","Bedroom light on","Bedroom lights are on now."),

]

In [26]:
# Preprocess the dataset
inputs = [tokenizer(question, return_tensors='tf', padding=True, truncation=True) for question, _, _ in dataset]
intent_labels = [intent for _, intent, _ in dataset]
chatbot_answers = [answer for _, _, answer in dataset]

In [28]:
# Convert intent labels into numerical representations
intent_label_map = {label: i for i, label in enumerate(set(intent_labels))}
intent_ids = [intent_label_map[label] for label in intent_labels]

In [29]:
# Prepare data for training
train_data = tf.data.Dataset.from_tensor_slices((inputs, intent_ids)).shuffle(len(dataset)).batch(32)

In [32]:
print(tf.__version__)

2.12.0


In [33]:
# Fine-tune the model (optional)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, epochs=10)

Epoch 1/10


AttributeError: in user code:

    File "c:\Users\aakas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\aakas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        
    File "c:\Users\aakas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        
    File "c:\Users\aakas\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_tf_utils.py", line 1571, in train_step
        x, y, sample_weight = keras.utils.unpack_x_y_sample_weight(data)

    AttributeError: module 'keras.utils' has no attribute 'unpack_x_y_sample_weight'


In [18]:
# Convert model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\aakas\AppData\Local\Temp\tmpyb3vp2pp\assets


INFO:tensorflow:Assets written to: C:\Users\aakas\AppData\Local\Temp\tmpyb3vp2pp\assets


KeyboardInterrupt: 